In [ ]:
%matplotlib notebook
%load_ext rpy2.ipython
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from pyvis.network import Network
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
readRDS = robjects.r['readRDS']


def get_num_edges(nodes):
    """Returns number of edges in a complete graph given number of nodes"""
    return nodes * (nodes - 1)/2

def get_num_nodes(edges):
    """Returns number of nodes in a complete graph given number of edges"""
    return 0.5 * (1 + np.sqrt(1 + 8*edges))

class ObjectiveValue(object):
    def __init__(self, filenames):
        self.__data__ = {x: getattr(self, f"__get_{x}__")(y) for x, y in filenames.items()}

    def __getitem__(self, key):
        return self.__data__[key]
    
    def get_data(self):
        return self.__data__
        
    def __get_cbc__(self, path_to_log):
        with open(path_to_log) as fh:
            obj_val = [x for x  in fh if "Objective value:" in x]
        return float(obj_val[-1].split("Objective value:")[-1].strip("\n "))
    
    def __get_cplex__(self, path_to_log):
        with open(path_to_log) as fh:
            obj_val = [x for x  in fh if "Objective = " in x]
        return float(obj_val[-1].split("Objective = ")[-1].strip("\n "))

    def __get_gurobi__(self, path_to_log):
        with open(path_to_log) as fh:
            obj_val = [x for x  in fh if "Best objective" in x]
        return float(obj_val[-1].split(",")[0].split("Best objective")[-1].strip("\n "))

def plot_benchmarks(value, edges_nodes, ilp_solvers, **kwargs):
    ni = 3 if "num_inputs" not in kwargs else kwargs["num_inputs"]
    nm = 3 if "num_meas" not in kwargs else kwargs["num_meas"]
    net_type = "Erdos" if "net_type" not in kwargs else kwargs["net_type"]
    figsize = (12, 5) if "figsize" not in kwargs else kwargs["figsize"]
    
    get_benchmark = lambda x, y, z : pd.read_csv(f"Output/{net_type}/E{x}_N{y}_I{ni}_M{nm}_S1_P2_2/{z}/benchmark.tsv", sep="\t")
    
    data = np.array([[get_benchmark(x, y, z)[value].values for x, y in edges_nodes] for z in ilp_solvers])
    
    if "scale" in kwargs: data = kwargs["scale"](data)
    x_ticklabels = [x[0] for x in edges_nodes]
    x_label = "Number of edges"
        
    fig, axs = plt.subplots(1, len(ilp_solvers), figsize=figsize, sharey=True)
    fig.suptitle(net_type)
    fig.subplots_adjust(bottom=0.2, left=0.1, right=0.9)
    for i, ax in enumerate(axs):
        ax.boxplot(data[i].tolist())
        ax.set_xticklabels(x_ticklabels)
        ax.set_xlabel(x_label)
        ax.set_title(ilp_solvers[i])
        
    if "ylabel" in kwargs:
        axs[0].set_ylabel(kwargs["ylabel"])
    if "file" in kwargs:
        fig.savefig(kwargs["file"])

        
time_settings = {"ilp_solvers": ["cplex", "cbc", "lpSolve"], 
                 "ylabel": "Execution time [mins]", 
                 "scale": lambda x : x/60,}
mem_settings = {"ilp_solvers": ["cplex", "cbc", "lpSolve"], 
                "ylabel": "Memory usage [GB]", 
                "scale": lambda x : x/1024,}
        
t_args1 = {**time_settings, 
           "edges_nodes": [[x, 50] for x in np.arange(200, 1000, 100)], 
           "file": "Figures/small_networks_time.svg"}
mem_args1 =  {**mem_settings,
              "edges_nodes": [[x, 50] for x in np.arange(200, 1000, 100)], 
              "file": "Figures/small_networks_memory.svg"}

t_args2 = {**time_settings, 
           "edges_nodes": [[x, 200] for x in np.arange(1000, 11000, 1000)], 
           "ilp_solvers": ["cplex", "cbc"],
           "file": "Figures/medium_networks_time.svg"}
mem_args2 =  {**mem_settings,
              "edges_nodes": [[x, 200] for x in np.arange(1000, 11000, 1000)], 
              "ilp_solvers": ["cplex", "cbc"],
              "file": "Figures/medium_networks_memory.svg"}

In [ ]:
num = np.arange(10, 1000, 10)
fig, ax = plt.subplots()
ax.plot(num, get_num_edges(num))
ax.set_title("Complete graph")
ax.set_xlabel("Number of nodes")
ax.set_ylabel("Number of edges")

# Visualising networks

Prior knowledge network (input for CARNIVAL):

In [ ]:
fname = "Output/Erdos/E600_N200_I3_M3_S1_P2_2/graph.dot"

g = nx.drawing.nx_agraph.read_dot(fname)
pos = nx.spring_layout(g, iterations=20)

fig, ax = plt.subplots()
nx.draw(g, pos, ax, with_labels=True)

In [ ]:
fname = "Output/Erdos/E300_N100_I3_M3_S1_P2_2/graph.dot"

g = Network(500, 1000, notebook=True)
g.from_DOT(fname)
g.show(fname.replace(".dot", ".html"))

In [ ]:
solvers = ("cbc", "cplex", "gurobi")

logs = {x: fname.replace("graph.dot", f"{x}/log.txt") for x in solvers}
obj_vals = ObjectiveValue(logs).get_data()

benchmarks = {x: pd.read_csv(fname.replace("graph.dot", f"{x}/benchmark.tsv"), "\t") 
              for x in solvers}

fig, axs = plt.subplots(1, 3, figsize=(12, 5))
fig.subplots_adjust(left=0.1, right=0.9, wspace=0.4)
axs[0].plot(solvers, [benchmarks[x]["s"].mean()/60 for x in solvers], "o")
axs[0].set_ylabel("Execution time [min]")
axs[1].plot(solvers, [benchmarks[x]["max_rss"].mean() for x in solvers], "o")
axs[1].set_ylabel("Memory usage [MB]")
axs[2].plot(solvers, [obj_vals[x] for x in solvers], "o")
axs[2].set_ylabel("Objective function value")

Solution given by CARNIVAL:

In [ ]:
solver = "cbc"
fname = fname.replace("graph.dot", f"{solver}/network_solution.dot")
g = Network(500, 1000, notebook=True)
g.from_DOT(fname)
g.show(fname.replace(".dot", ".html"))

In [ ]:
%%R
print(readRDS("Output/Erdos/E100_N20_I3_M3_S1_P2_2/cplex/result.Rds")$weightedSIF)
print(readRDS("Output/Erdos/E100_N20_I3_M3_S1_P2_2/cbc/result.Rds")$weightedSIF)
print(readRDS("Output/Erdos/E100_N20_I3_M3_S1_P2_2/lpSolve/result.Rds")$weightedSIF)